In [53]:
import numpy as np
from tensorflow.keras.models import load_model
import os
import matplotlib.pyplot as plt
from scipy.stats import norm
from tensorflow.keras.preprocessing import image
import cv2

In [54]:
#可变参数
model_folder_name='saved generator_13'

In [55]:
#固定参数
n = 50 #每个维度取值数量
latent_dim = 2
img_shape = (128, 512, 1) #对于1920*1080电影截屏图片，第一个1920是列（图片水平向像素数量）、第二个1080是行（图片竖向像素数量）。所以输入神经网络时，input_shape=(720 * 1980,)
png_folder='.\\GAN - %dD_img\\'%(latent_dim)

#加载模型
generator = load_model('.\\%s\\'%(model_folder_name))  # 载入模型
generator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              6144      
_________________________________________________________________
reshape (Reshape)            (None, 2, 8, 128)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 4, 16, 128)        262272    
_________________________________________________________________
batch_normalization (BatchNo (None, 4, 16, 128)        512       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 4, 16, 128)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 4, 16, 128)        0   

In [56]:
#《python深度学习》P255代码清单8-28。
figure = np.zeros((img_shape[0] * n, img_shape[1] * n,3)) #因为要添加文字，故RGB格式
for max_value in [100,10,1]: #每个维度取值的最大值
    grid_x = grid_y =np.linspace(-max_value, max_value, n)#.np.linspace(0, 1, 11)=[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
    for i, xi in enumerate(grid_x): 
        for j, yi in enumerate(grid_y):
            z_sample = np.array([[xi, yi]])
            x_decoded = generator.predict(z_sample) #tanh值(-1,1)
            digit = x_decoded[0]
            figure[j * img_shape[0]: (j + 1) * img_shape[0],i * img_shape[1]: (i + 1) * img_shape[1],:] = digit
            info='x=%.1f,y=%.1f'%(xi, yi) #文字内容
            p_center=(int(i * img_shape[1]+img_shape[1]*1/3),int(j * img_shape[0]+img_shape[0]*5/6)) #文字位置
            cv2.putText(figure,info,p_center,cv2.FONT_HERSHEY_PLAIN,1,(1,1,1),1) #color=(1,1,1)的原因：保存文件时要乘以255。《OpenCV 4.5计算机视觉开发实战（基于Python）》P81
    figure=figure * 127.5 + 127.5 #因为tanh值(-1,1)
    img = image.array_to_img(figure, scale=False)
    img.save(os.path.join(png_folder, 'model_folder_name=%s,max_value=%d.png'%(model_folder_name,max_value)))